<a href="https://colab.research.google.com/github/manjitullal/unsupervised/blob/main/Manjit_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 PageRank

In this exercise we will compute the PageRank scores for pages in a dataset provided by
Google in a programming contest in 2002. The file web-Google_10k.txt contains data about 10,000 web pages and 78,323 links. The file format is:

* The first four rows contain metadata about the data set and are self-explained.
* The following rows consist of two values that represent the link from the web page in the first column to the web page in the second column.
For example, if the row is 0 11342, this means that there is a directed link from page id
0 to page id 11342.

Answer the following questions:
1. Find all the dead ends. A page is a dead end if it has no out-going links.
2. Build the page transition matrix A from the data set and make sure it is a stochastic
matrix.
3. Implement the PageRank algorithm.
4. Run the PageRank algorithm on the matrix A with a damping factor of d = 0:85 and
$\epsilon$ = 0:001. The output format should be two-column: the first column is the web page
id and the second column is its PageRank score. Sort the output by descending order
of the PageRank scores.

# Solution:

1. The number of deadend nodes are 1235
2. transition matrix explained below
3. implemented below
4. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/datasets/other/'
! ls $path

web-Google_10k.txt


In [3]:
with open(path + 'web-Google_10k.txt') as file:
  data = file.readlines()

In [4]:
sample_graph = [[1, 3],
         [1, 2],
         [2, 3],
         [2, 1],
         [3, 2],
         [4, 3],
         [4, 5],
         [4, 6],
         [6, 4],
         [6, 5]]

In [5]:
sample_graph2 = [['a', 'b'],
                 ['a', 'c'],
                 ['a', 'd'],
                 ['b', 'a'],
                 ['b', 'd'],
                 ['c', 'c'],
                 ['d', 'b'],
                 ['d', 'c']]

create a 2D list for the graph like above graph

In [6]:
google_graph = []
for i in range(4, len(data)):
  row = data[i].strip()
  row = row.split('\t')
  google_graph.append(row)

In [7]:
google_graph[0:4]

[['0', '11342'], ['0', '824020'], ['0', '867923'], ['0', '891835']]

In [8]:
import numpy as np
import pandas as pd

create a dictionary which represents the graph, where the key is the node and the values are the list of nodes that have edges going from key

In [9]:
def create_graph(graph):
  dict_graph = {}
  for node in graph:
    if node[0] in dict_graph:
      dict_graph[node[0]].extend([node[1]])
    else:
      dict_graph[node[0]] = [node[1]]
  return dict_graph

In [11]:
dict_sample_graph = create_graph(sample_graph)
dict_google_graph = create_graph(google_graph)

print(create_graph(sample_graph))

{1: [3, 2], 2: [3, 1], 3: [2], 4: [3, 5, 6], 6: [4, 5]}


In [12]:
dict_sample_graph2 = create_graph(sample_graph2)
print(dict_sample_graph2)

{'a': ['b', 'c', 'd'], 'b': ['a', 'd'], 'c': ['c'], 'd': ['b', 'c']}


In [38]:
# union operation using sets 
unique_sample_nodes = list(set(sum(dict_sample_graph.values(), [])) | set(dict_sample_graph.keys()))
unique_sample_nodes = sorted(unique_sample_nodes, key = int)
unique_google_nodes = list(set(sum(dict_google_graph.values(), [])) | set(dict_google_graph.keys()))
unique_google_nodes = sorted(unique_google_nodes, key = int)

In [39]:
unique_sample_nodes2 = list(set(sum(dict_sample_graph2.values(), [])) | set(dict_sample_graph2.keys()))
unique_sample_nodes2.sort()

In [40]:
print("Sample Graph")
print("Number of unique nodes: ", len(unique_sample_nodes), "\n" ,"Unique nodes: ", unique_sample_nodes)
print("Google Graph")
print("Number of unique nodes: ", len(unique_google_nodes), "\n" ,"Unique nodes: ", unique_google_nodes[0:5])

Sample Graph
Number of unique nodes:  6 
 Unique nodes:  [1, 2, 3, 4, 5, 6]
Google Graph
Number of unique nodes:  10000 
 Unique nodes:  ['0', '1', '2', '4', '5']


### 1. Transition Matrix

create a transition matrix where the value for each cell value is (1/outbound_links_node)

deadend nodes get value as (1/totalnodesin_graph)

In [41]:
def create_adjacency_matrix(unique_nodes, dict_graph, n):
  a,b = 0,0  
  dead_end = 0
  adjacency_matrix = np.zeros(shape=(n,n))
  for i in unique_nodes:
    b = 0
    for j in unique_nodes:
      adjacent = dict_graph.get(i, [])
      if j in adjacent:
        adjacency_matrix[a,b] = 1/len(dict_graph[i])            
      else:
        if len(adjacent) == 0:
          dead_end +=1 
          adjacency_matrix[a,b] = 1/len(unique_nodes)
      b += 1
    a += 1
  print("Number of deadends nodes: ", dead_end/n)
  return adjacency_matrix

adjacency_sample_matrix = create_adjacency_matrix(unique_sample_nodes, dict_sample_graph, len(unique_sample_nodes))
adjacency_google_matrix = create_adjacency_matrix(unique_google_nodes, dict_google_graph, len(unique_google_nodes))

Number of deadends nodes:  1.0
Number of deadends nodes:  1235.0


In [42]:
print('\n')
print(adjacency_sample_matrix) # visualization only for sample matrix as original matrix is very big



[[0.         0.5        0.5        0.         0.         0.        ]
 [0.5        0.         0.5        0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.        ]
 [0.         0.         0.33333333 0.         0.33333333 0.33333333]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.         0.         0.         0.5        0.5        0.        ]]


In [83]:
# adjacency_sample_matrix2 = create_adjacency_matrix(unique_sample_nodes2, dict_sample_graph2, len(unique_sample_nodes2))
# print(adjacency_sample_matrix2)

augment the matrix to make it stochastic

In [22]:
#@title
# def augmented_adjacency_matrix(adjacency_matrix, d):
#   return d*adjacency_matrix.transpose() + (1-d)/adjacency_matrix.shape[0]
# augmented_adjacency_sample_matrix2 = augmented_adjacency_matrix(adjacency_sample_matrix2, 0.8)
# print(augmented_adjacency_sample_matrix2)
# augmented_adjacency_sample_matrix = augmented_adjacency_matrix(adjacency_sample_matrix, 0.9)
# print(augmented_adjacency_sample_matrix)
# augmented_adjacency_google_matrix = augmented_adjacency_matrix(adjacency_google_matrix, 0.9)
# #print(augmented_adjacency_google_matrix)

power iteration method to find the page rank vector

In [74]:
def power_iteration(matrix, d, epsilon):
  p_o_prev = np.ones(shape=(matrix.shape[0],1))/matrix.shape[0] 
  p_k_prev = d*matrix.transpose() + (1-d)/matrix.shape[0]
  error = np.inf
  while error > epsilon:
    p_k = d*matrix.transpose()*p_k_prev + (1-d)/matrix.shape[0]
    Ax = np.dot(p_k, p_o_prev)
    norm = np.linalg.norm(Ax)
    p_o = Ax/norm
    error = np.sqrt(np.sum((p_o_prev - p_o)**2, axis=0))
    p_o_prev = p_o
    p_k_prev = p_k
  return p_o

In [75]:
#matrix = np.array([[3, 2], [2, 6]])
epsilon = 0.001

#sample_eigen_vector = power_iteration(adjacency_sample_matrix2, 0.8, epsilon)
google_eigen_vector = power_iteration(adjacency_google_matrix, 0.85, epsilon)
#print(sample_eigen_vector)

In [76]:
final_output = {'Id': unique_google_nodes, 'PageRank': google_eigen_vector.flatten()}
final_output = pd.DataFrame(final_output)

In [82]:
final_output[final_output.Id == '486980']

,Id,PageRank
5187,486980,0.011975


In [88]:
final_output.sort_values(by='PageRank', inplace=True)
final_output.head(10)

,Id,PageRank
206,243,0.009994
142,171,0.009994
89,109,0.009994
143,172,0.009994
144,173,0.009994
146,175,0.009994
248,297,0.009994
149,179,0.009994
246,295,0.009994
80,97,0.009994


# 2 Recommender Systems
Consider the following matrix of user ratings on a 5-star scale:

> | User| Item 1 | Item 2 | Item 3 | Item 4 | Item 5 | Item 6
>--- | --- | --- | ---| --- | --- | ---
>User A | 5 | 3 | 4 |  |  | 1
>User B | 4 | 1 | 2 | 2 |  | 1
>User C | 1 |  |  | 3 | 1 | 2
>User D | 1 | 5 | 2 |  | 5 | 
>User E | 1 |  | 3 | 1 | 3 | 


1. Predict the rating of Item 4 by User A using user-user CF with Pearson correlation coefficient and neighborhood size of 2. Show your computation.

2. Predict the rating of Item 4 by User A using item-item CF with cosine similarity and neighborhood size of 3. Show your computation.

In [ ]:
# import pandas as pd
# ratings = {'user': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E'],
#            'item': ['item1', 'item2', 'item3', 'item6', 'item1', 'item2', 'item3', 'item4', 'item6', 'item1', 'item4', 'item5', 'item6'
#            'item1', 'item2', 'item3', 'item5', 'item1', 'item3', 'item4', 'item5'],
#            'rating': [5, 3, 4, 1, 4, 1, 2, 2, 1, 1, 3, 1, 2, 1, 5, 2, 5, 1, 3, 1, 3] }
# ratings_df = pd.DataFrame(ratings)
# ratings_df.head()